# Prediction of S&P in relation to the previous day's values and gold prices using Linear Regression

In [62]:
import time 
import datetime
import pandas as pd
import yfinance as yf
import numpy as np

In [63]:
df_gold = pd.read_excel('stock_data/GoldPrices.xlsx',sheet_name='Daily',header=8,index_col=3,parse_dates=True)[['US dollar']]
#df_gold.index

In [64]:
#Find the begining and end of gold dataset
start = df_gold.index[0]  # start
period1 = int(start.timestamp()) # begining 

end = df_gold.index[-1] # end
period2 = int(end.timestamp()) # begining 

interval = '1d'
symbol_snp = '%5EGSPC'# SP500 symbol
symbol_dow = '^DJI' #Dow Jones symbol

In [65]:
querr_snp = f'https://query1.finance.yahoo.com/v7/finance/download/{symbol_snp}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
df_snp = pd.read_csv(querr_snp,index_col=0,header=0,parse_dates=True)
df_snp.to_csv(f'stock_data/{symbol_snp}.csv')
df_snp = pd.read_csv(f'stock_data/{symbol_snp}.csv',index_col=0,header=0,parse_dates=True)
df_snp.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1978-12-29,0.0,97.029999,95.480003,96.110001,96.110001,30030000
1979-01-02,0.0,96.959999,95.220001,96.730003,96.730003,18340000
1979-01-03,0.0,98.540001,96.809998,97.800003,97.800003,29180000
1979-01-04,0.0,99.419998,97.519997,98.580002,98.580002,33290000
1979-01-05,0.0,99.790001,98.250000,99.129997,99.129997,28890000


In [66]:
querr_dow = f'https://query1.finance.yahoo.com/v7/finance/download/{symbol_snp}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
df_dow = pd.read_csv(querr_snp,index_col=0,header=0,parse_dates=True)
df_snp.to_csv(f'stock_data/{symbol_snp}.csv')
df_snp = pd.read_csv(f'stock_data/{symbol_snp}.csv',index_col=0,header=0,parse_dates=True)
df_snp.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1978-12-29,0.0,97.029999,95.480003,96.110001,96.110001,30030000
1979-01-02,0.0,96.959999,95.220001,96.730003,96.730003,18340000
1979-01-03,0.0,98.540001,96.809998,97.800003,97.800003,29180000
1979-01-04,0.0,99.419998,97.519997,98.580002,98.580002,33290000
1979-01-05,0.0,99.790001,98.250000,99.129997,99.129997,28890000


In [67]:
df_snp.loc[df_snp['Open']==0,'Open'] = df_snp['Close'].shift(1) # if the Open Value is zero replace it with yesterday's Close
df_snp.loc['1978-12-29','Open'] = df_snp.loc['1978-12-29','Low']# for the first day replace it with the low of the day
        
df_snp.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1978-12-29,95.480003,97.029999,95.480003,96.110001,96.110001,30030000
1979-01-02,96.110001,96.959999,95.220001,96.730003,96.730003,18340000
1979-01-03,96.730003,98.540001,96.809998,97.800003,97.800003,29180000
1979-01-04,97.800003,99.419998,97.519997,98.580002,98.580002,33290000
1979-01-05,98.580002,99.790001,98.250000,99.129997,99.129997,28890000


In [68]:
#Remove rows that are not in snp, which means dates that the stock market was closed
df_gold = df_gold.loc[df_snp.index]
df_gold

,US dollar
Date,
1978-12-29,226.00
1979-01-02,226.80
1979-01-03,218.60
1979-01-04,223.15
1979-01-05,225.50
...,...
2022-10-14,1649.30
2022-10-17,1664.75
2022-10-18,1653.00


In [69]:
from sklearn import preprocessing
from sklearn import linear_model

In [70]:
n = int(len(df_snp)*0.8)
x_train = df_snp.iloc[:n-1][['Open','High','Low','Close','Volume']]
x_train['Gold'] = df_gold.iloc[:n-1]['US dollar']
x_train['Open'] = np.where(x_train['Open']==0,x_train['Low'],x_train['Open']) # if the value of open is zero then replace it with the low, it actually should be yesterday's close
y_train = df_snp.iloc[1:n][['Open','High','Low','Close']]
y_train['Open'] = np.where(y_train['Open']==0,y_train['Low'],y_train['Open'])

In [71]:
x_train.head()

,Open,High,Low,Close,Volume,Gold
Date,,,,,,
1978-12-29,95.480003,97.029999,95.480003,96.110001,30030000,226.00
1979-01-02,96.110001,96.959999,95.220001,96.730003,18340000,226.80
1979-01-03,96.730003,98.540001,96.809998,97.800003,29180000,218.60
1979-01-04,97.800003,99.419998,97.519997,98.580002,33290000,223.15
1979-01-05,98.580002,99.790001,98.250000,99.129997,28890000,225.50


In [72]:
x_test = df_snp.iloc[n:-1][['Open','High','Low','Close','Volume']]
x_test['Gold'] = df_gold.iloc[n:-1]['US dollar']
x_test['Open'] = np.where(x_test['Open']==0,x_test['Low'],x_test['Open'])
y_test = df_snp.iloc[n+1:][['Open','High','Low','Close']]
y_test['Open'] = np.where(y_test['Open']==0,y_test['Low'],y_test['Open'])


In [73]:
reg = linear_model.LinearRegression().fit(x_train,y_train)
results = pd.DataFrame(data=reg.coef_,columns=['Open','High','Low','Close','Volume','Gold'],index=['Open','High','Low','Close'])
results["Intercept"] = reg.intercept_
results

,Open,High,Low,Close,Volume,Gold,Intercept
Open,0.005948,-0.002062,-0.000333,0.996520,-4.468778e-11,0.000109,-0.056813
High,0.038408,0.274076,-0.226809,0.916664,5.273434e-10,-0.001395,0.718556
Low,-0.032241,-0.249799,0.236109,1.043235,-8.543180e-10,0.002518,-0.981168
Close,-0.041256,0.163828,0.042896,0.834080,-3.387651e-10,0.001323,-0.292727


In [74]:
y_pred = pd.DataFrame(data=reg.predict(x_test),columns=['Open','High','Low','Close'], index=y_test.index)
(y_pred-y_test).head()

,Open,High,Low,Close
Date,,,,
2014-01-14,-2.026618,-6.605942,-15.607439,-17.455777
2014-01-15,-1.675729,-3.513582,-10.957233,-10.070679
2014-01-16,0.384282,7.854203,-0.214782,2.107294
2014-01-17,1.726750,6.918609,2.935395,6.540312
2014-01-21,-2.280117,-2.411956,-2.398091,-4.973116


In [75]:
score = 1-np.sum((y_pred-y_test)**2,axis=0)/np.sum((y_test-y_test.mean(axis=0))**2,axis=0)
print('\n The below table shows R2 linear regression for each entry of the next day:\n',score)


 The below table shows R2 linear regression for each entry of the next day:
 Open     0.999674
High     0.999222
Low      0.998840
Close    0.998286
dtype: float64


In [76]:
mse = np.sum((y_pred-y_test)**2,axis=0)/len(y_pred)
mse

Open      223.338778
High      541.324232
Low       782.396645
Close    1174.632010
dtype: float64

## Use the percentage change instead of the actual value

In [77]:
x_train_change = (x_train.pct_change()*100).dropna() # remove NAN 
y_train_change = (y_train.pct_change()*100).dropna()
x_test_change = (x_test.pct_change()*100).dropna()
y_test_change = (y_test.pct_change()*100).dropna()

In [78]:
reg_change = linear_model.LinearRegression().fit(x_train_change,y_train_change)
results_change = pd.DataFrame(reg_change.coef_,columns=['Open','High','Low','Close','Volume','Gold'],index=['Open','High','Low','Close'])
results_change["Intercept"] = reg.intercept_
results_change

,Open,High,Low,Close,Volume,Gold,Intercept
Open,0.004704,-0.013465,-0.005329,0.982313,-0.000006,-0.001725,-0.056813
High,0.186567,-0.495330,0.012707,0.686101,-0.000212,0.002233,0.718556
Low,0.191754,-0.003616,-0.480941,0.799426,0.000145,0.009522,-0.981168
Close,-0.083255,0.044956,0.027363,-0.062267,-0.000285,0.007497,-0.292727


In [79]:
y_pred_change = pd.DataFrame(data=reg_change.predict(x_test_change),columns=['Open','High','Low','Close'], index=y_test_change.index)
(y_pred_change-y_test_change).head()

,Open,High,Low,Close
Date,,,,
2014-01-15,0.007590,0.050905,-0.528629,-0.448236
2014-01-16,0.095615,0.421965,0.126447,0.102341
2014-01-17,0.075533,0.188753,0.273013,0.405550
2014-01-21,-0.208403,-0.411656,-0.037211,-0.201248
2014-01-22,0.074007,0.218524,-0.191730,-0.021931


In [80]:
score = 1-np.sum((y_pred_change-y_test_change)**2,axis=0)/np.sum((y_test_change-y_test_change.mean(axis=0))**2,axis=0)
print('\n The below table shows R2 linear regression for each entry of the next day:\n',score)


 The below table shows R2 linear regression for each entry of the next day:
 Open     0.453598
High     0.087605
Low      0.035051
Close    0.008204
dtype: float64


In [81]:
print( 'Predited Price by the First Method\n')
print(y_pred.tail())

print( '\nPredited Price by the Second Method\n')
y_pred_2nd = y_pred_change/100*y_test.shift(1)+y_test.shift(1)
print(y_pred_2nd.tail())

Predited Price by the First Method

                   Open         High          Low        Close
Date                                                          
2022-10-14  3669.213322  3718.511256  3617.741620  3669.496662
2022-10-17  3583.637757  3632.332769  3536.492280  3598.450852
2022-10-18  3677.902670  3697.875408  3656.574640  3678.588047
2022-10-19  3720.252760  3749.785707  3689.865549  3723.168126
2022-10-20  3695.348920  3720.433974  3669.350698  3697.842951

Predited Price by the Second Method

                   Open         High          Low        Close
Date                                                          
2022-10-14  3609.370713  3697.977841  3589.616696  3672.132012
2022-10-17  3604.651809  3674.026631  3502.172871  3579.302003
2022-10-18  3733.011366  3759.686934  3678.131498  3678.665761
2022-10-19  3787.658200  3777.492263  3717.949741  3714.202246
2022-10-20  3679.322808  3720.762789  3648.728962  3699.478993
